<a href="https://colab.research.google.com/github/EAsencios/DEEP-LEARING/blob/master/data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Data Wrangling**

In [2]:
import random
import numpy as np
import pandas as pd

import datetime
from random import randrange

In [3]:
from time import strftime
startDate = datetime.datetime(2020, 1, 1,8)


def _random_date(start,date_count):
    """This function generates a random date based on params
    Args:
        start (date object): the base date
        date_count (int): number of dates to be generated
    Returns:
        list of random dates

    """
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1


def generate_sample_data(row_count=100):
    """This function generates a random transaction dataset
    Args:
        row_count (int): number of rows for the dataframe
    Returns:
        a pandas dataframe

    """

    # sentinels
    startDate = datetime.datetime(2020, 1, 1, 8)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000

    data_dict = {
    'Serial No': np.arange(row_count)+serial_number_sentinel,
    'Date': np.random.permutation(pd.to_datetime([x.strftime('%d-%m-%Y') for x in _random_date(startDate,row_count)]).date),
    'User ID': np.random.permutation(np.random.randint(0, row_count, size=int(row_count/10)) + user_id_sentinel).tolist()*10,
    'Product ID': np.random.permutation(np.random.randint(0, row_count, size=int(row_count/10))+ product_id_sentinel).tolist()*10 ,
    'Quantity Purchased': np.random.permutation(np.random.randint(1, 42, size=row_count)),
    'Price': np.round(np.abs(np.random.randn(row_count)+1)*price_sentinel, decimals=2),
    'User Type':np.random.permutation([chr(random.randrange(97, 97 + 3 + 1)) for i in range(row_count)])
    }

    
    # introduce missing values
    for index in range(int(np.sqrt(row_count))): 
        data_dict['Price'][np.argmax(data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(data_dict['User Type'] == random.choice(data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(data_dict['User ID']))] = -101
        

    df = pd.DataFrame(data_dict)

    return(df)


def describe_dataframe(df=pd.DataFrame()):
    """This function generates descriptive stats of a dataframe
    Args:
        df (dataframe): the dataframe to be analyzed
    Returns:
        None

    """
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)

    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
        
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
       
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).to_numpy().nonzero()[0].tolist()))
    print("\n")

    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).to_numpy().nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")

    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))
    
    return


def cleanup_column_names(df,rename_dict={},do_inplace=True):
    """This function renames columns of a pandas dataframe
       It converts column names to snake case if rename_dict is not passed. 
    Args:
        rename_dict (dict): keys represent old column names and values point to 
                            newer ones
        do_inplace (bool): flag to update existing dataframe or return a new one
    Returns:
        pandas dataframe if do_inplace is set to False, None otherwise

    """
    if not rename_dict:
        return df.rename(columns={col: col.lower().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=do_inplace)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

def expand_user_type(u_type):
    """This function maps user types to user classes
    Args:
        u_type (str): user type value
    Returns:
        (str) user_class value

    """
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'


In [4]:
df = generate_sample_data(1000)

In [5]:
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int64
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int64
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 62


Sample Indices with missing data:: [2, 4, 6, 10, 11]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Serial No           1000 non-null   int64  
 1   Date                969 non-null    object 
 2   User ID             1000 non-null   int64  
 3   P

,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,2020-10-02,-101,0,30,4549.16,n
1,1001,2020-01-15,5711,277,20,1758.15,n
2,1002,NaN,5453,955,26,2899.79,n
3,1003,2020-01-21,5797,337,39,3733.89,n
4,1004,NaN,5846,233,21,2081.87,n


In [6]:
print('Dataframe columns:\n{}'.format(df.columns.tolist()))

Dataframe columns:
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


In [7]:
cleanup_column_names(df)

In [8]:
print('Dataframe columns:\n{}'.format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


In [9]:
print("Using Column Name::")
df['quantity_purchased'].values[:10]

Using Column Name::


array([30, 20, 26, 39, 21, 41, 41, 31,  2,  9])

In [10]:
print("Using Column Name::" )
df.quantity_purchased.values[:10]

Using Column Name::


array([30, 20, 26, 39, 21, 41, 41, 31,  2,  9])

In [11]:
print("Using Column Data Type::" )
df.select_dtypes(include=['float64']).values[:10]

Using Column Data Type::


array([[4549.16],
       [1758.15],
       [2899.79],
       [3733.89],
       [2081.87],
       [4358.92],
       [1198.15],
       [ 226.03],
       [1099.73],
       [1740.61]])

In [12]:
print('Select Specific row indices::')
df.iloc[[20, 30, 50, 60]]

Select Specific row indices::


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
20,1020,NaN,5912,967,31,3078.37,b
30,1030,2020-01-01,5603,555,15,2543.88,b
50,1050,2020-01-25,5389,571,26,1212.65,b
60,1060,2020-09-02,5813,794,37,6353.78,d


In [13]:
print('Excluding Specific Row indices::')
df.drop([1, 14, 21], axis=0).head(20)

Excluding Specific Row indices::


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
0,1000,2020-10-02,-101,0,30,4549.16,n
2,1002,NaN,5453,955,26,2899.79,n
3,1003,2020-01-21,5797,337,39,3733.89,n
4,1004,NaN,5846,233,21,2081.87,n
5,1005,2020-11-02,5321,155,41,4358.92,n
6,1006,NaN,5973,632,41,1198.15,n
7,1007,2020-04-02,5004,525,31,226.03,n
8,1008,2020-02-02,5866,323,2,1099.73,n
9,1009,2020-03-01,5611,655,9,1740.61,n
10,1010,NaN,5453,746,22,1203.34,n


In [14]:
print('Subsetting based on logical condition(s)::')
df[df.quantity_purchased>25].head()

Subsetting based on logical condition(s)::


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
0,1000,2020-10-02,-101,0,30,4549.16,n
2,1002,NaN,5453,955,26,2899.79,n
3,1003,2020-01-21,5797,337,39,3733.89,n
5,1005,2020-11-02,5321,155,41,4358.92,n
6,1006,NaN,5973,632,41,1198.15,n


In [15]:
print('Subsetting based on offset from top (bottom)::')
df[100:].head()  # df.tail(-100)

Subsetting based on offset from top (bottom)::


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,NaN,5384,124,3,1727.80,d
101,1101,2020-11-02,5711,277,34,3117.96,d
102,1102,2020-07-01,5453,955,13,4691.55,c
103,1103,2020-01-18,5797,337,23,472.65,a
104,1104,NaN,5846,233,3,1605.37,b


In [16]:
df.tail(-100)

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,NaN,5384,124,3,1727.80,d
101,1101,2020-11-02,5711,277,34,3117.96,d
102,1102,2020-07-01,5453,955,13,4691.55,c
103,1103,2020-01-18,5797,337,23,472.65,a
104,1104,NaN,5846,233,3,1605.37,b
...,...,...,...,...,...,...,...
995,1995,2020-12-01,5392,245,11,2492.10,a
996,1996,2020-03-02,5141,824,24,2289.53,d
997,1997,2020-12-01,5628,421,16,1978.17,b
998,1998,2020-01-23,5645,783,41,2610.74,d


In [17]:
df['date'] = pd.to_datetime(df.date)
df.dtypes

serial_no                      int64
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int64
price                        float64
user_type                     object
dtype: object

In [18]:
df['user_class'] = df['user_type'].map(expand_user_type)
df

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
0,1000,2020-10-02,-101,0,30,4549.16,n,error
1,1001,2020-01-15,5711,277,20,1758.15,n,error
2,1002,NaT,5453,955,26,2899.79,n,error
3,1003,2020-01-21,5797,337,39,3733.89,n,error
4,1004,NaT,5846,233,21,2081.87,n,error
...,...,...,...,...,...,...,...,...
995,1995,2020-12-01,5392,245,11,2492.10,a,new
996,1996,2020-03-02,5141,824,24,2289.53,d,loyal_existing
997,1997,2020-12-01,5628,421,16,1978.17,b,new
998,1998,2020-01-23,5645,783,41,2610.74,d,loyal_existing


In [20]:
df['purchase_week'] = df[['date']].applymap(lambda dt:dt.week if not pd.isnull(dt.week) else 0)
df

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2020-10-02,-101,0,30,4549.16,n,error,40
1,1001,2020-01-15,5711,277,20,1758.15,n,error,3
2,1002,NaT,5453,955,26,2899.79,n,error,0
3,1003,2020-01-21,5797,337,39,3733.89,n,error,4
4,1004,NaT,5846,233,21,2081.87,n,error,0
...,...,...,...,...,...,...,...,...,...
995,1995,2020-12-01,5392,245,11,2492.10,a,new,49
996,1996,2020-03-02,5141,824,24,2289.53,d,loyal_existing,10
997,1997,2020-12-01,5628,421,16,1978.17,b,new,49
998,1998,2020-01-23,5645,783,41,2610.74,d,loyal_existing,4


In [30]:
df.select_dtypes(include=[np.number]).apply(lambda x: x.max() - x.min())

serial_no             2000.00
user_id               6074.00
product_id            1083.00
quantity_purchased      40.00
price                 8990.39
purchase_week           49.00
dtype: float64

In [33]:
print('Drop Row with missing dates::')
df_dropped = df.dropna(subset=['date'])
print("Shape::", df_dropped.shape)
df_dropped

Drop Row with missing dates::
Shape:: (969, 9)


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2020-10-02,-101,0,30,4549.16,n,error,40
1,1001,2020-01-15,5711,277,20,1758.15,n,error,3
3,1003,2020-01-21,5797,337,39,3733.89,n,error,4
5,1005,2020-11-02,5321,155,41,4358.92,n,error,45
7,1007,2020-04-02,5004,525,31,226.03,n,error,14
...,...,...,...,...,...,...,...,...,...
995,1995,2020-12-01,5392,245,11,2492.10,a,new,49
996,1996,2020-03-02,5141,824,24,2289.53,d,loyal_existing,10
997,1997,2020-12-01,5628,421,16,1978.17,b,new,49
998,1998,2020-01-23,5645,783,41,2610.74,d,loyal_existing,4


In [37]:
print('Fill Missing Price values with mean price:::')
df_dropped['price'].fillna(value=np.round(df.price.mean(), decimals=2), inplace=False)

Fill Missing Price values with mean price:::


0      4549.16
1      1758.15
3      3733.89
5      4358.92
7       226.03
        ...   
995    2492.10
996    2289.53
997    1978.17
998    2610.74
999    2358.04
Name: price, Length: 969, dtype: float64

In [38]:
print('Fill Missing user_type values with value from \ previous row (forward fill)::')
df_dropped['user_type'].fillna(method='ffill', inplace=False)

Fill Missing user_type values with value from \ previous row (forward fill)::


0      n
1      n
3      n
5      n
7      n
      ..
995    a
996    d
997    b
998    d
999    c
Name: user_type, Length: 969, dtype: object

In [39]:
print('Fill Missing user_type values with valve from \ next row (backward fill)::')
df_dropped['user_type'].fillna(method='bfill', inplace=False)

Fill Missing user_type values with valve from \ next row (backward fill)::


0      n
1      n
3      n
5      n
7      n
      ..
995    a
996    d
997    b
998    d
999    c
Name: user_type, Length: 969, dtype: object